Seguindo o que foi ensinado no outro arquivo:

In [18]:
#Aula 07 - Música e som
#Peguei os sons de:
#https://themushroomkingdom.net/media/smw/wav
#https://freemusicarchive.org/member/meghan-admin/meet-zapa-mix/


from cmath import sqrt
import pygame
from pygame.locals import *
from sys import exit
from random import randint # Importa a função que gera números aleatórios

pygame.init()

musicaDeFundo = pygame.mixer.music.load('Zapa - Beiramar.mp3')
pygame.mixer.music.set_volume(0.1) #Valores entre 0 e 1
pygame.mixer.music.play(-1)

somColisao = pygame.mixer.Sound('smw_coin.wav')
somColisao.set_volume(1)

largura = 640
altura = 480

nCol = 3
nLin = 3

numeroPecas = 16
raiz= sqrt(numeroPecas)

if raiz.real.is_integer():
    nCol = int(raiz.real)
    nLin = int(raiz.real)

tela = pygame.display.set_mode((largura, altura), 0, 32)

x_mouse = 0
y_mouse = 0

x_cobra = int(largura/2)
y_cobra = int(altura/2)

x_comida = int(randint(40, 600))
y_comida = int(randint(50, 430))

tempo_cobra = 500
velocidade = 10
x_controle = velocidade
y_controle = 0

x_puzzle_begin = 40
y_puzzle_begin = 50

x_puzzle_end = 300
y_puzzle_end = 430

peca_largura = int((x_puzzle_end - x_puzzle_begin)/nLin)
peca_altura = int((y_puzzle_end - y_puzzle_begin)/nCol)

pontos = 0
fonte = pygame.font.SysFont("lucidaconsole", 20, True, True)

# Cores
branco = (255, 255, 255)
preto = (0, 0, 0)
vermelho = (255, 0, 0)
azul = (0, 0, 255)
verde = (0, 255, 0)
corFundo = (200, 80, 80)
# corObjetos = (10, 50, 0)
corObjetos = preto

# Criando clock
clock = pygame.time.Clock()

# Altera o título da janela
pygame.display.set_caption("Olá Mundo!")

lista_pecas = []

lista_cobra = []
comprimento_inicial = 5
morreu = False

x_peca_dist = (x_puzzle_end - x_puzzle_begin)/nLin
y_peca_dist = (y_puzzle_end - y_puzzle_begin)/nCol

matriz_coordenadas = []
matriz_pecas = []
matriz_sprites = []
peca = []

# Inicialização das matrizes
for i in range(nCol):
    linha = []
    linha_pecas = []
    linha_sprites = []
    
    for j in range(nLin):
        x = x_puzzle_begin + j * x_peca_dist
        y = y_puzzle_begin + i * y_peca_dist
        linha.append((x, y))
        
        x_p = int(randint(300, 600))
        y_p = int(randint(50, 430))
        linha_pecas.append((x_p, y_p))
        linha_sprites.append(f"{i}x{j}")
        
    matriz_coordenadas.append(linha)
    matriz_pecas.append(linha_pecas)
    matriz_sprites.append(linha_sprites)

print(matriz_sprites)

# Carregar os sprites
sprites = []
for i in range(nCol):
    linha_sprites = []
    for j in range(nLin):
        try:
            sprite_path = f'4x4/{matriz_sprites[i][j]}.jpg'
            print(sprite_path)
            sprite_ = pygame.image.load(sprite_path).convert_alpha()
            sprite_ = pygame.transform.scale(sprite_, (peca_largura, peca_altura))
            linha_sprites.append(sprite_)
        except pygame.error as e:
            print(f"Erro ao carregar sprite {sprite_path}: {e}")
            linha_sprites.append(None)
    sprites.append(linha_sprites)

# Verifique se os sprites foram carregados corretamente
for i in range(nCol):
    for j in range(nLin):
        if sprites[i][j] is None:
            print(f"Sprite não carregado em {i}, {j}")
    
print(matriz_sprites)

pecaSegueMouse = False
MUDAR_DIRECAO_EVENT = pygame.USEREVENT + 1
pygame.time.set_timer(MUDAR_DIRECAO_EVENT, tempo_cobra)
    
    
def aumentaCobra(lista_cobra):
    for XeY in lista_cobra:
        pygame.draw.rect(tela, corObjetos, (XeY[0], XeY[1], 20, 20))

def reiniciar_jogo():
    global pontos, comprimento_inicial, x_cobra, y_cobra, x_comida, y_comida, lista_cobra, lista_cabeca, x_controle, y_controle, morreu
    pontos = 0
    comprimento_inicial = 5
    x_cobra = int(largura/2)
    y_cobra = int(altura/2)
    x_comida = int(randint(40, 600))
    y_comida = int(randint(50, 430))
    lista_cobra = []
    lista_cabeca = []

    morreu = False 


    
while True:
    clock.tick(30)
    tela.fill(corFundo)
    mensagem = f'Pontos: {pontos}'
    textoFormatado = fonte.render(mensagem, True, branco)
    
    for event in pygame.event.get():
        if event.type == QUIT:
            pygame.quit()
            exit()
        
        if event.type == KEYDOWN:
            if event.key == K_a:
                if x_controle == velocidade:
                    pass
                else:
                    x_controle = -velocidade
                    y_controle = 0
            if event.key == K_d:
                if x_controle == -velocidade:
                    pass
                else:
                    x_controle = velocidade
                    y_controle = 0
            if event.key == K_w:
                if y_controle == velocidade:
                    pass
                else:
                    y_controle = -velocidade
                    x_controle = 0
            if event.key == K_s:
                if y_controle == -velocidade:
                    pass
                else:
                    y_controle = velocidade
                    x_controle = 0
                    
        if event.type == MUDAR_DIRECAO_EVENT:
            tempo_cobra = randint(500, 2000)
            x_controle = randint(-1, 1) * velocidade
            y_controle = randint(-1, 1) * velocidade
            if x_controle == 0 and y_controle == 0:
                x_controle = velocidade
                y_controle = 0
                
        if event.type == MOUSEBUTTONDOWN:
            x_mouse, y_mouse = event.pos
            if not pecaSegueMouse:
                for i in range(nCol):
                    for j in range(nLin):
                        x_peca, y_peca = matriz_pecas[i][j]
                        if x_mouse >= x_peca and x_mouse <= (x_peca + x_peca_dist) and y_mouse >= y_peca and y_mouse <= (y_peca + y_peca_dist):
                            pecaSegueMouse = True
                            clicouNaPeca = True
                            peca_selecionada = (i, j)
                            print("Clicou em", i, j, "---", pecaSegueMouse, clicouNaPeca)
                            break
                    if pecaSegueMouse:
                        break
            elif clicouNaPeca:
                pecaSegueMouse = False
                clicouNaPeca = False
                i, j = peca_selecionada
                print("Soltou em", i, j),  "---", pecaSegueMouse, clicouNaPeca
                if abs(matriz_pecas[i][j][0] - matriz_coordenadas[i][j][0]) < 10 and abs(matriz_pecas[i][j][1] - matriz_coordenadas[i][j][1]) < 10:
                    matriz_pecas[i][j] = matriz_coordenadas[i][j]    
    
        if event.type == MOUSEMOTION and pecaSegueMouse:
            x_mouse, y_mouse = event.pos
            x_peca = x_mouse - x_peca_dist / 2
            y_peca = y_mouse - y_peca_dist / 2
            
            peca_rect = pygame.Rect(x_peca, y_peca, x_peca_dist, y_peca_dist)
            print("Mouse em", x_mouse, y_mouse)
            print("Peca em", x_peca, y_peca)
            for i in range(nCol):
                    for j in range(nLin):
                        if i == peca_selecionada[0] and j == peca_selecionada[1]:
                            matriz_pecas[i][j] = (x_peca, y_peca)
                            
                            if cobra.colliderect(peca_rect) and pecaSegueMouse:
                                pecaSegueMouse = False
                                clicouNaPeca = False
                                pontos -= 1
                                velocidade += 1
                                matriz_pecas[i][j] = (int(randint(40, 600)), int(randint(50, 430)))
                                somColisao.play()
                                print("Colisão em", i, j,  "---", pecaSegueMouse, clicouNaPeca)
                                aumentaCobra(lista_cobra)
                                break
                        if not pecaSegueMouse:
                            break

    
    x_cobra += x_controle
    y_cobra += y_controle
            
    puzzle = pygame.draw.rect(tela, branco, (x_puzzle_begin, y_puzzle_begin, x_puzzle_end - x_puzzle_begin, y_puzzle_end - y_puzzle_begin), 1)
    
    for i in range(nCol):
        for j in range(nLin):
            x_peca, y_peca = matriz_pecas[i][j]
            x, y = matriz_coordenadas[i][j]
            pygame.draw.rect(tela, branco, (x, y, x_peca_dist, y_peca_dist), 1)
            if sprites[i][j] is not None:
                tela.blit(sprites[i][j], (x_peca, y_peca))
    
    cobra = pygame.draw.rect(tela, corObjetos, (x_cobra, y_cobra, 20, 20))
    lista_cabeca = []
    lista_cabeca.append(x_cobra)   
    lista_cabeca.append(y_cobra)    
    lista_cobra.append(lista_cabeca)

    
    if lista_cobra.count(lista_cabeca) > 1 and clicouNaPeca == True:
        fonte2 = pygame.font.SysFont("Arial", 30, True, True)
        mensagem2 = 'Você morreu! Pressione R para reiniciar'
        textoFormatado2 = fonte2.render(mensagem2, True, branco)
        retTexto = textoFormatado2.get_rect()
        
        # morreu = True
        while morreu:
            tela.fill(preto)
            for event in pygame.event.get():
                if event.type == QUIT:
                    pygame.quit()
                    exit()
                if event.type == KEYDOWN:
                    if event.key == K_r:
                       reiniciar_jogo()
                       
            retTexto.center = (largura//2, altura//2)               
            tela.blit(textoFormatado2, retTexto)               
            pygame.display.update()
            
    if x_cobra > largura:
        x_cobra = 0
    if x_cobra < 0:
        x_cobra = largura
    if y_cobra < 0:
        y_cobra = altura
    if y_cobra > altura:
        y_cobra = 0
        
    
    if len(lista_cobra) > comprimento_inicial:
        del lista_cobra[0]
    
    aumentaCobra(lista_cobra)
    
    tela.blit(textoFormatado, (450, 40))
    pygame.display.update()

[['0x0', '0x1', '0x2', '0x3'], ['1x0', '1x1', '1x2', '1x3'], ['2x0', '2x1', '2x2', '2x3'], ['3x0', '3x1', '3x2', '3x3']]
4x4/0x0.jpg
4x4/0x1.jpg
4x4/0x2.jpg
4x4/0x3.jpg
4x4/1x0.jpg
4x4/1x1.jpg
4x4/1x2.jpg
4x4/1x3.jpg
4x4/2x0.jpg
4x4/2x1.jpg
4x4/2x2.jpg
4x4/2x3.jpg
4x4/3x0.jpg
4x4/3x1.jpg
4x4/3x2.jpg
4x4/3x3.jpg
[['0x0', '0x1', '0x2', '0x3'], ['1x0', '1x1', '1x2', '1x3'], ['2x0', '2x1', '2x2', '2x3'], ['3x0', '3x1', '3x2', '3x3']]
Clicou em 2 1 --- True True
Mouse em 408 207
Peca em 375.5 159.5
Mouse em 349 207
Peca em 316.5 159.5
Mouse em 301 208
Peca em 268.5 160.5
Mouse em 296 210
Peca em 263.5 162.5
Mouse em 293 212
Peca em 260.5 164.5
Mouse em 289 214
Peca em 256.5 166.5
Mouse em 282 221
Peca em 249.5 173.5
Mouse em 273 229
Peca em 240.5 181.5
Mouse em 269 231
Peca em 236.5 183.5
Mouse em 264 234
Peca em 231.5 186.5
Mouse em 263 235
Peca em 230.5 187.5
Mouse em 262 239
Peca em 229.5 191.5
Mouse em 265 256
Peca em 232.5 208.5
Mouse em 281 274
Peca em 248.5 226.5
Mouse em 289 281
Pec

SystemExit: 